In [1]:
# Import findspark and initialize.
import findspark
findspark.init()
print("FindSpark initialized successfully!")

FindSpark initialized successfully!


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 13:43:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to Spark's context
spark.sparkContext.addFile(url)

# Load the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the first few rows to verify the data
df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [7]:
# 2. Create a temporary view of the DataFrame.
# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")

In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT YEAR(date) AS year_sold, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# Execute the query
result_df = spark.sql(query)

# Show the results
result_df.show()


+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT YEAR(date) AS year_built, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
"""

# Execute the query
result_df = spark.sql(query)

# Show the results
result_df.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2019|287287.82|
|      2020|294204.16|
|      2021|294211.46|
|      2022|292725.69|
+----------+---------+



In [13]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT YEAR(date) AS year_built, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 
  AND bathrooms = 3 
  AND floors = 2 
  AND sqft_living >= 2000
GROUP BY year_built
ORDER BY year_built
"""

# Execute the query
result_df = spark.sql(query)

# Show the results
result_df.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2019|289859.14|
|      2020|292289.09|
|      2021|296330.96|
|      2022|290242.99|
+----------+---------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
import time

# Start timer
start_time = time.time()
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0001709461212158203 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.
# Cache the home_sales temporary table
spark.sql("CACHE TABLE home_sales")

# Confirm caching
print("Temporary table 'home_sales' has been cached.")


Temporary table 'home_sales' has been cached.


In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""
# Show results
result_df.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2019|289859.14|
|      2020|292289.09|
|      2021|296330.96|
|      2022|290242.99|
+----------+---------+

--- 0.16463422775268555 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Define the output path for the partitioned Parquet file
# Change output path to a directory inside your home directory
output_path = "/Users/andrewmeza/Desktop/Module_22/home_sales_partitioned"

# Save the DataFrame as a Parquet file partitioned by "date_built"
df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")


# Save the DataFrame as a Parquet file partitioned by "date_built"
df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")

print("Partitioned Parquet file has been successfully saved!")

Partitioned Parquet file has been successfully saved!


In [27]:
# 11. Read the formatted parquet data.
# Define the path where the partitioned Parquet file was saved
parquet_path = "/Users/andrewmeza/Desktop/Module_22/home_sales_partitioned"

# Read the partitioned Parquet data
df_parquet = spark.read.parquet(parquet_path)

# Show the first few rows to verify
df_parquet.show(5)


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [29]:
# 12. Create a temporary table for the parquet data.
# Create a temporary SQL view
df_parquet.createOrReplaceTempView("home_sales_parquet")

print("Temporary table 'home_sales_parquet' has been created successfully!")

# Show available tables
spark.catalog.listTables()


Temporary table 'home_sales_parquet' has been created successfully!


[Table(name='home_sales', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='home_sales_parquet', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [31]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
# Run the query using the Parquet DataFrame
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query
result_df = spark.sql(query)

# Show results
result_df.show()

# End timer and print execution time
print("--- %s seconds ---" % (time.time() - start_time))

CodeCache: size=131072Kb used=35824Kb max_used=35824Kb free=95247Kb
 bounds [0x0000000105f08000, 0x0000000108238000, 0x000000010df08000]
 total_blobs=13743 nmethods=12641 adapters=1011
 compilation: disabled (not enough contiguous free space left)
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.3379690647125244 seconds ---


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


In [33]:
# 14. Uncache the home_sales temporary table.
spark.catalog.isCached("home_sales")

True

In [35]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' still cached? {is_cached}")

Is 'home_sales' still cached? True
